In [1]:
import os
import shutil

# [Transfer Learning을 위한 준비]

import torch

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

BATCH_SIZE = 256
EPOCH = 30

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

data_transforms = {
    'train':transforms.Compose([
        transforms.Resize([64,64]),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],
                             [0.229,0.224,0.225])
    ]),
    'val':transforms.Compose([
        transforms.Resize([64,64]),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],
                             [0.229,0.224,0.225])
    ])
}

data_dir = '/home/iai/Desktop/Jeewon/Study/PyTorch/PyTorchProject/data/splitted'

image_datasets = {x: ImageFolder(root=os.path.join(data_dir,x),
                                 transform=data_transforms[x]) for x in ['train','val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size = BATCH_SIZE, shuffle=True, num_workers = 4) for x in ['train','val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [2]:
# [Pre-Trained Model 불러오기]

from torchvision import models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler

resnet = models.resnet50(pretrained=True)
# 이 프로젝트에선 33개로 분류해야함. 마지막 Layer 출력 채널 수는 33개여야함.
# 하지만 ResNet의 마지막 Layer 출력 채널 수가 33개가 아님. 이 프로젝트의 주제에 맞추고자 모델의 마지막 Fully Connected Layer 대신 출력 채널의 수가 33개인 새로운 Layer를 추가할 예정
# in_features : 해당 Layer의 입력 채널 수
num_ftrs = resnet.fc.in_features
# 불러온 모델의 마지막 Fully Connected Layer를 새로운 Layer로 교체. 입력 채널의 수는 기존 Layer와 동일하고, 출력 채널 수는 33개로 수정
resnet.fc = nn.Linear(num_ftrs, 33)
resnet = resnet.to(DEVICE)

criterion = nn.CrossEntropyLoss()

# Transfer Learning에선 일부 Layer의 Parameter만을 업데이트 해야하기 때문에 requires_grad = True로 설정된 Layer의 Parameter에만 적용
optimizer_ft = optim.Adam(filter(lambda p:p.requires_grad, resnet.parameters()), lr = 0.001)

# StepLR() 메서드는 Epoch에 따라 Learning Rate를 변경하는 역할
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
# Transfer Learning에서는 불러오는 모델의 Parameter 업데이트 부분을 지정할 수 있음.
# Transfer Learning은 기본적으로 Fully Connected Layer 앞 부분의 Parameter를 고정시키고 뒷부분만 학습시키므로, 고정시킬 Parameter를 지정해주어야함.
# 이 부분을 설정하지 않으면 모델은 처음부터 학습하게 됨

In [3]:
# [Pre-Trained Model 일부 Layer Freeze 하기]

ct = 0 # 해당 Layer가 몇 번째인지
for child in resnet.children(): # resnet.children()은 생성한 resnet모델의 모든 Layer 정보를 담고 있음
    ct += 1 # 다음 Layer를 지칭하도록 ct변수를 1씩 증가
    if ct < 6: # 1~5번 Layer는 업데이트 되지 않도록 고정. 6~10번 Layer는 업데이트
        for param in child.parameters():
            param.requires_grad = False


In [4]:
# [Transfer Learning 모델 학습과 검증을 위한 함수]

import time
import copy

def train_resnet(model, criterion, optimizer, scheduler, num_epochs=25):

    best_model_wts = copy.deepcopy(model.state_dict())  
    best_acc = 0.0  
    
    for epoch in range(num_epochs):
        print('-------------- epoch {} ----------------'.format(epoch+1)) 
        since = time.time()                                     
        for phase in ['train', 'val']: 
            if phase == 'train': 
                model.train()  # 학습 모드
            else:
                model.eval() # 검증 모드
 
            running_loss = 0.0  
            running_corrects = 0  
 
            
            for inputs, labels in dataloaders[phase]: 
                inputs = inputs.to(DEVICE)  
                labels = labels.to(DEVICE)  
                
                optimizer.zero_grad() 
                
                with torch.set_grad_enabled(phase == 'train'): # 학습 단계에서만 모델 Gradient 업데이트, 검증 단계에서는 X
                    outputs = model(inputs)  
                    _, preds = torch.max(outputs, 1) 
                    loss = criterion(outputs, labels)  
    
                    if phase == 'train':   
                        loss.backward()
                        optimizer.step()
 
                running_loss += loss.item() * inputs.size(0)  
                running_corrects += torch.sum(preds == labels.data)  
            if phase == 'train':  
                scheduler.step()
 
            epoch_loss = running_loss/dataset_sizes[phase]  
            epoch_acc = running_corrects.double()/dataset_sizes[phase]  
 
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc)) 
 
          
            if phase == 'val' and epoch_acc > best_acc: 
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
 
        time_elapsed = time.time() - since  
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
 
    model.load_state_dict(best_model_wts) 

    return model

In [5]:
model_resnet50 = train_resnet(resnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=EPOCH) 

torch.save(model_resnet50, 'resnet50.pt')

-------------- epoch 1 ----------------
train Loss: 0.5987 Acc: 0.8185
val Loss: 0.2732 Acc: 0.9145
Completed in 0m 10s
-------------- epoch 2 ----------------
train Loss: 0.2253 Acc: 0.9271
val Loss: 0.1737 Acc: 0.9423
Completed in 0m 9s
-------------- epoch 3 ----------------
train Loss: 0.1593 Acc: 0.9476
val Loss: 0.1555 Acc: 0.9509
Completed in 0m 9s
-------------- epoch 4 ----------------
train Loss: 0.1366 Acc: 0.9557
val Loss: 0.1584 Acc: 0.9506
Completed in 0m 9s
-------------- epoch 5 ----------------
train Loss: 0.1161 Acc: 0.9611
val Loss: 0.1520 Acc: 0.9521
Completed in 0m 9s
-------------- epoch 6 ----------------
train Loss: 0.1000 Acc: 0.9664
val Loss: 0.1440 Acc: 0.9542
Completed in 0m 9s
-------------- epoch 7 ----------------
train Loss: 0.0860 Acc: 0.9721
val Loss: 0.1198 Acc: 0.9627
Completed in 0m 9s
-------------- epoch 8 ----------------
train Loss: 0.0442 Acc: 0.9850
val Loss: 0.0526 Acc: 0.9839
Completed in 0m 9s
-------------- epoch 9 ----------------
train L

In [6]:
# [Transfer Learning모델 평가를 위한 전처리]

transform_resNet = transforms.Compose([
        transforms.Resize([64,64]),  
        transforms.RandomCrop(52),  
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
    ])
    
test_resNet = ImageFolder(root='/home/iai/Desktop/Jeewon/Study/PyTorch/PyTorchProject/data/splitted/test', transform=transform_resNet) 
test_loader_resNet = torch.utils.data.DataLoader(test_resNet, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [7]:
# [모델 평가를 위한 함수]

def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad(): # torch.no_grad() : Parameter 업데이트 중단
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction = 'sum').item()
            
            pred = output.max(1, keepdim=True)[1] # Test 데이터가 33개의 클래스에 속할 각각의 확률값이 output으로 출력. 가장 높은 가진 인덱스를 예측값으로 저장
            correct += pred.eq(target.view_as(pred)).sum().item() # pred와 target.view_as(pred)가 일치하면 1, 일치하지 않으면 0 반환
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [8]:
# [Transfer Learning 모델 성능 평가하기]

resnet50=torch.load('resnet50.pt') 
resnet50.eval()  
test_loss, test_accuracy = evaluate(resnet50, test_loader_resNet)

print('ResNet test acc:  ', test_accuracy)

ResNet test acc:   98.88596820628364
